In [1]:
import pandas as pd 

In [2]:
data = pd.read_parquet("D:/Orange/Episafe_local/data/parquet/train/1869/006/UTC-2019_11_18-03_20_00.parquet")

In [3]:
def change_frequency(df):
    df['timestamp'] = pd.to_datetime(df['utc_timestamp'], unit='s')

    # Set the timestamp column as the DataFrame index
    df.set_index('timestamp', inplace=True)

    # Create a new DataFrame with a datetime index at 4 Hz intervals
    resampled_df = pd.DataFrame(index=pd.date_range(start=df.index.min(), end=df.index.max(), freq='250L'))

    # Use groupby to resample the original data to 4 Hz (128 Hz / 32 = 4 Hz)
    for column in df.columns:
        resampled_df[column] = df[column].groupby(pd.Grouper(freq='250L')).mean()
    
    resampled_df['date_time'] = resampled_df.index
    resampled_df['timestamp_column'] = resampled_df['date_time'].apply(lambda x: pd.Timestamp(x).timestamp())
    resampled_df.reset_index(inplace=True)
    resampled_df.drop(['utc_timestamp','date_time','index'],axis=1,inplace=True)
    return resampled_df

    

In [4]:
resampled_data2 = change_frequency(data)

In [5]:
resampled_data2

,acc_x,acc_y,acc_z,acc_mag,bvp,eda,hr,temp,timestamp_column
0,55.712984,-11.997415,27.396944,62.233745,-26.836599,2.385498,83.795565,35.469970,1.574047e+09
1,54.891088,-11.997415,28.132993,61.835910,-86.526334,2.375839,83.783154,35.469970,1.574047e+09
2,54.850816,-11.872165,28.887325,62.122016,-39.079332,2.364943,83.770742,35.471146,1.574047e+09
3,54.497571,-11.997415,29.001159,61.889129,-11.767519,2.352350,83.758298,35.485661,1.574047e+09
4,54.512511,-11.756087,29.528096,62.103908,-21.304289,2.338461,83.762380,35.489998,1.574047e+09
...,...,...,...,...,...,...,...,...,...
2395,6.143971,4.852654,62.263901,61.758334,-197.498760,0.843120,84.004147,35.432316,1.574048e+09
2396,8.419479,4.373833,62.361209,62.135063,-12.091709,0.861917,84.056205,35.429978,1.574048e+09
2397,7.178327,4.768270,62.232536,61.841970,159.185101,0.862791,84.116101,35.429978,1.574048e+09
2398,6.098307,4.875650,62.271240,61.768771,376.966070,0.863820,84.175186,35.435292,1.574048e+09


In [6]:
resampled_data2.to_csv('2400_data.csv')